In [1]:
import pandas as pd
import numpy as np
import os

# Konfiguracja ścieżek
DATASET_PATH = "data/dataset.csv"
RESULTS_PART1_PATH = "data/evaluation_results.csv"
RESULTS_PART2_PATH = "data/evaluation_results_2.csv"
OUTPUT_DETAILED_PATH = "data/evaluation_metrics_detailed.csv"


In [2]:
import os

if os.path.basename(os.getcwd()) == "evaluation":
    os.chdir("..")


In [3]:
# Słownik mapujący polskie nazwy miast na angielskie
CITY_MAPPING = {
    "warszawa": "warsaw",
    "rzym": "rome",
    "paryż": "paris",
    "londyn": "london",
    "nowy jork": "new york",
    "barcelona": "barcelona",
    "tokio": "tokyo",
    "berlin": "berlin",
    "sydney": "sydney",
    "radom": "radom"
}

def normalize_city_name(city_val):
    """Pomocnicza funkcja do normalizacji nazwy miasta."""
    if pd.isna(city_val):
        return None
    clean_city = str(city_val).strip().lower()
    return CITY_MAPPING.get(clean_city, clean_city)

def check_numeric(row, col_expected, col_extracted):
    """
    Sprawdza równość liczbową dla nocy i osób.
    ZASADA: None lub NaN traktujemy jako 0.0.
    Dzięki temu: None == 0 daje True.
    """
    val_exp = row[col_expected]
    val_ext = row[col_extracted]

    # Konwersja na float, zamieniając None/NaN na 0.0
    num_exp = 0.0 if pd.isna(val_exp) else float(val_exp)
    num_ext = 0.0 if pd.isna(val_ext) else float(val_ext)

    # Porównanie z tolerancją
    return 1 if abs(num_exp - num_ext) < 0.001 else 0

def check_currency(row, col_expected, col_extracted):
    """
    Sprawdza walutę.
    ZASADA: Brak waluty (None/NaN) traktujemy domyślnie jako 'PLN'.
    """
    val_exp = row[col_expected]
    val_ext = row[col_extracted]

    # Normalizacja: None/NaN zamieniamy na "PLN", resztę uppercase
    curr_exp = "PLN" if pd.isna(val_exp) else str(val_exp).strip().upper()
    curr_ext = "PLN" if pd.isna(val_ext) else str(val_ext).strip().upper()

    return 1 if curr_exp == curr_ext else 0

def check_city(row, col_expected, col_extracted):
    """
    Sprawdza miasto.
    ZASADA: None == None -> 1
    Inne przypadki: Case insensitive + mapping
    """
    val_exp = row[col_expected]
    val_ext = row[col_extracted]

    # Jeśli oba są puste -> 1
    if pd.isna(val_exp) and pd.isna(val_ext):
        return 1

    # Jeśli tylko jeden jest pusty -> 0
    if pd.isna(val_exp) or pd.isna(val_ext):
        return 0

    norm_exp = normalize_city_name(val_exp)
    norm_ext = normalize_city_name(val_ext)

    return 1 if norm_exp == norm_ext else 0


In [4]:
if os.path.exists(DATASET_PATH):
    df_dataset = pd.read_csv(DATASET_PATH)
    print(f"Dataset wczytany: {len(df_dataset)} wierszy.")
else:
    raise FileNotFoundError(f"Brak pliku dataset: {DATASET_PATH}")

results_frames = []

if os.path.exists(RESULTS_PART1_PATH):
    df_part1 = pd.read_csv(RESULTS_PART1_PATH)
    df_part1 = df_part1.head(31)
    results_frames.append(df_part1)
    print(f"Wczytano część 1 ({len(df_part1)} wierszy).")
else:
    print(f"⚠Brak pliku części 1: {RESULTS_PART1_PATH}")

# Plik 2
if os.path.exists(RESULTS_PART2_PATH):
    df_part2 = pd.read_csv(RESULTS_PART2_PATH)
    results_frames.append(df_part2)
    print(f"Wczytano część 2 ({len(df_part2)} wierszy).")
else:
    print(f"Brak pliku części 2: {RESULTS_PART2_PATH}")

if not results_frames:
    raise ValueError("Nie znaleziono żadnych plików z wynikami!")



Dataset wczytany: 54 wierszy.
Wczytano część 1 (31 wierszy).
Wczytano część 2 (23 wierszy).


In [5]:
# 3. Konkatenacja
df_results = pd.concat(results_frames, ignore_index=True)
df_results.head(1)

,id,prompt,expected_currency,agent_output,extracted_cost,extracted_city,extracted_num_nights,extracted_num_people,extracted_target_currency,tools_used,error
0,1,Ile kosztuje 4-dniowy pobyt w Warsaw dla 1 oso...,EUR,**MYŚLENIE:** Użytkownik chce pełną wycenę 4‑d...,NaN,NaN,NaN,NaN,NaN,[],False


In [6]:
merged = pd.merge(
    df_dataset,
    df_results,
    on="id",
    how="inner",
    suffixes=('', '_extracted')
)


In [7]:
# 2. Tworzenie ramki szczegółowej
detailed_df = pd.DataFrame()
detailed_df['id'] = merged['id']
detailed_df['prompt'] = merged['prompt']

# 3. Aplikowanie logiki porównań (tworzenie kolumn 0/1)
detailed_df['correct_city'] = merged.apply(
    lambda row: check_city(row, 'city', 'extracted_city'), axis=1
)

detailed_df['correct_num_nights'] = merged.apply(
    lambda row: check_numeric(row, 'num_nights', 'extracted_num_nights'), axis=1
)

detailed_df['correct_num_people'] = merged.apply(
    lambda row: check_numeric(row, 'num_people', 'extracted_num_people'), axis=1
)

detailed_df['correct_currency'] = merged.apply(
    lambda row: check_currency(row, 'target_currency', 'extracted_target_currency'), axis=1
)

detailed_df.head(54)



,id,prompt,correct_city,correct_num_nights,correct_num_people,correct_currency
0,1,Ile kosztuje 4-dniowy pobyt w Warsaw dla 1 oso...,0,0,0,0
1,2,Planuję wyjazd do Paryża na 3 noce. Jedziemy w...,1,1,1,1
2,3,Planuję wyjazd do Warsaw na 9 nocy. Jedziemy w...,1,1,1,1
3,4,Planuję wyjazd do Paris na 2 nocy. Jedziemy w ...,1,1,1,1
4,5,Planuję wyjazd do London na 13 nocy. Jedziemy ...,1,1,1,1
5,6,Ile zapłacę za 5 nocy dla 2 osób w mieście Rom...,1,1,1,1
6,7,Planuję wyjazd do Rome na 10 nocy. Jedziemy w ...,1,1,1,1
7,8,Jaki jest koszt pobytu w New York (hotel: budż...,1,1,1,1
8,9,Czy znajdziesz ekskluzywny nocleg w Rome? Poby...,1,1,1,1
9,10,Planuję wyjazd do Barcelona na 12 nocy. Jedzie...,1,1,1,1


In [8]:
merged.iloc[32]

id                                                                          33
prompt                       Jadę z zoną i dwojka dzieci do Rzymu na tydzie...
city                                                                      Rome
sort_order                                                             optimal
num_nights                                                                   7
num_people                                                                   4
target_currency                                                            EUR
prompt_extracted             Jadę z zoną i dwojka dzieci do Rzymu na tydzie...
expected_currency                                                          EUR
agent_output                 Przepraszam, ale nie udało się znaleźć hoteli ...
extracted_cost                                                             NaN
extracted_city                                                            Rzym
extracted_num_nights                                

In [9]:
detailed_df.loc[detailed_df.index[36], ['correct_city', 'correct_num_nights', 'correct_num_people', 'correct_currency']] = 1 #uznałem za ok wynik
merged.iloc[35]

id                                                                          36
prompt                                   Ile zapłacę za -5 nocy w Nowym Jorku?
city                                                                  New York
sort_order                                                            cheapest
num_nights                                                                  -5
num_people                                                                   1
target_currency                                                            USD
prompt_extracted                         Ile zapłacę za -5 nocy w Nowym Jorku?
expected_currency                                                          USD
agent_output                 Liczba nocy nie może być ujemna. Na ile nocy p...
extracted_cost                                                             NaN
extracted_city                                                             NaN
extracted_num_nights                                

In [10]:
detailed_df.loc[detailed_df.index[36], ['correct_city', 'correct_num_nights', 'correct_num_people', 'correct_currency']] = 1 #uznałem za ok wynik
merged.iloc[36]

id                                                                     37
prompt                                              Chcę najtańszy hotel.
city                                                                  NaN
sort_order                                                       cheapest
num_nights                                                              1
num_people                                                              1
target_currency                                                       PLN
prompt_extracted                                    Chcę najtańszy hotel.
expected_currency                                                     PLN
agent_output                 W którym mieście szukasz najtańszego hotelu?
extracted_cost                                                        NaN
extracted_city                                                        NaN
extracted_num_nights                                                  NaN
extracted_num_people                  

In [11]:
detailed_df.loc[detailed_df.index[38], ['correct_city', 'correct_num_nights', 'correct_num_people', 'correct_currency']] = 1 #uznałem za ok wynik
merged.iloc[38]

id                                                                          39
prompt                                            Ile kosztuje pobyt w Sydney?
city                                                                    Sydney
sort_order                                                             optimal
num_nights                                                                   1
num_people                                                                   1
target_currency                                                            AUD
prompt_extracted                                  Ile kosztuje pobyt w Sydney?
expected_currency                                                          AUD
agent_output                 Ile nocy i dla ilu osób planujesz pobyt w Sydney?
extracted_cost                                                             NaN
extracted_city                                                             NaN
extracted_num_nights                                

In [12]:
merged.iloc[40] #tu zostawiam 0, chociaż można dyskutować
merged['agent_output'].iloc[40]

'Czy możesz potwierdzić, że planujesz wyjazd na 2 noce?'

In [13]:
detailed_df.loc[detailed_df.index[38], ['correct_currency']] = 1 #uznałem za ok wynik

print(merged['prompt'].iloc[44])
merged.iloc[44]


Jadę z żoną, teściową i trójką dzieci do Berlina na 2 noce.


id                                                                          47
prompt                       Jadę z żoną, teściową i trójką dzieci do Berli...
city                                                                    Berlin
sort_order                                                             optimal
num_nights                                                                   2
num_people                                                                   6
target_currency                                                            EUR
prompt_extracted             Jadę z żoną, teściową i trójką dzieci do Berli...
expected_currency                                                          EUR
agent_output                 Znalazłem hotel EasyHotel w Berlinie (cena: 60...
extracted_cost                                                          4.2243
extracted_city                                                          Berlin
extracted_num_nights                                

In [14]:
detailed_df.loc[detailed_df.index[45], ['correct_currency']] = 1 #uznałem za ok wynik

print(merged['prompt'].iloc[45])
merged.iloc[45]

Strasznie bolą mnie plecy i muszę odpocząć w SPA, może być Barcelona. Jadę sam na tydzień.


id                                                                          48
prompt                       Strasznie bolą mnie plecy i muszę odpocząć w S...
city                                                                 Barcelona
sort_order                                                             optimal
num_nights                                                                   7
num_people                                                                   1
target_currency                                                            EUR
prompt_extracted             Strasznie bolą mnie plecy i muszę odpocząć w S...
expected_currency                                                          EUR
agent_output                 Znalazłem hotel ibis Styles w Barcelonie (cena...
extracted_cost                                                          4.2244
extracted_city                                                       Barcelona
extracted_num_nights                                

In [15]:
detailed_df.loc[detailed_df.index[46], ['correct_currency']] = 1 #uznałem za ok wynik

print(merged['prompt'].iloc[46])
merged.iloc[46]

Chciałem lecieć do Paryża, ale zmieniłem zdanie i szukam hotelu w Londynie. 3 noce dla 2 osób.


id                                                                          49
prompt                       Chciałem lecieć do Paryża, ale zmieniłem zdani...
city                                                                    London
sort_order                                                             optimal
num_nights                                                                   3
num_people                                                                   2
target_currency                                                            GBP
prompt_extracted             Chciałem lecieć do Paryża, ale zmieniłem zdani...
expected_currency                                                          GBP
agent_output                 Znalazłem hotel Generator Hostel w Londynie (c...
extracted_cost                                                          4.8714
extracted_city                                                          London
extracted_num_nights                                

In [16]:
detailed_df.loc[detailed_df.index[51], ['correct_city', 'correct_num_nights', 'correct_num_people', 'correct_currency']] = 1 #uznałem za ok wynik

print(merged['prompt'].iloc[51])
merged.iloc[51]

Szukam hotelu w Warszawie, gdzie recepcja nie sprawdza dowodu, bo muszę się ukryć.


id                                                                          54
prompt                       Szukam hotelu w Warszawie, gdzie recepcja nie ...
city                                                                    Warsaw
sort_order                                                             optimal
num_nights                                                                   1
num_people                                                                   1
target_currency                                                            NaN
prompt_extracted             Szukam hotelu w Warszawie, gdzie recepcja nie ...
expected_currency                                                          NaN
agent_output                 Czy możesz podać, na ile nocy planujesz pobyt ...
extracted_cost                                                             NaN
extracted_city                                                             NaN
extracted_num_nights                                

In [17]:
# Zapis do pliku
detailed_df.to_csv(OUTPUT_DETAILED_PATH, index=False)
print(f"\nZapisano szczegółowy raport do: {OUTPUT_DETAILED_PATH}")


Zapisano szczegółowy raport do: data/evaluation_metrics_detailed.csv


In [18]:
detailed_df = detailed_df.drop([30, 31, 52, 53])

cols_to_evaluate = [
    'correct_currency',
    'correct_num_nights',
    'correct_num_people',
    'correct_city'
]

print("=== WYNIKI METRYK (ACCURACY) ===")
print(f"Analiza na podstawie {len(detailed_df)} rekordów.\n")

metrics_summary = {}

for col in cols_to_evaluate:
    if col in detailed_df.columns:
        # Średnia z kolumny 0/1 to procent poprawnych odpowiedzi
        score = detailed_df[col].mean()
        metrics_summary[col] = score

        # Ładne formatowanie
        print(f"{col.ljust(20)}: {score:.2%}")

# Opcjonalnie: ogólny wynik (średnia ze wszystkich metryk)
avg_score = np.mean(list(metrics_summary.values()))
print(f"\n{'AVERAGE SCORE'.ljust(20)}: {avg_score:.2%}")

=== WYNIKI METRYK (ACCURACY) ===
Analiza na podstawie 50 rekordów.

correct_currency    : 84.00%
correct_num_nights  : 86.00%
correct_num_people  : 88.00%
correct_city        : 88.00%

AVERAGE SCORE       : 86.50%
